In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

c:\Users\lutz\miniconda3\envs\hcss\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
def summarize(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [19]:
with open("data/processed/2022-01-18-eerstekamer-8.txt", "r") as file:
    text = file.read()

In [12]:
print(summarize(text))

the supplementary budget for the acquisition of artwork has failed. the cultural sector is fighting for its survival and manyzzp'ers in that sector live below the poverty line. this is also important for non-culture miners, but also for the sector as an employer.


topic detection

In [24]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="albert-base-v2") # Or "albert-base-v2"
candidate_topics = ["defence", "security", "geopolitics", "climate", "energy", "technology"]
result = classifier(text, candidate_topics)
print(result)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


{'sequence': 'Bruijn, Jan Anthonie (VVD): The next item is the debate on the legislative proposal 35984, Amendment of the budget statement of the Ministry of Education, Culture and Science for the year 2022, the incidental supplementary budget for the acquisition of artwork. I would like to welcome the Secretary of State for Culture and Media to the First Chamber. You are here in the First Chamber as the first ruler of the new cabinet. I wish you every success in your work as Secretary of State.\nI give the floor to Mrs Veldhoen on behalf of the Group of the Greens/European Free Alliance.\nVeldhoen, Gala (GL): Thank you, Mr. Chairman. It is good that we are having this debate today, on the first Chamber day of this new year. I just heard that after hundreds of years of Binnenhof this is the first meeting out there. It is also the first debate of this new ruler. It\'s a special moment. We are pleased with a cultural historian in this place. I would therefore like to welcome her to this 

In [25]:
print(result['labels'])
print(result['scores'])

['geopolitics', 'technology', 'climate', 'energy', 'security', 'defence']
[0.17406964302062988, 0.16985245048999786, 0.16809895634651184, 0.16525328159332275, 0.16312257945537567, 0.1596030592918396]


In [4]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3")

with open("data/processed/debates/2022-01-19-tweedekamer-3.txt", "r") as file:
    debate_text = file.read()

candidate_topics = ["Economy", "healthcare", "education", "environment"]  # Replace with your topics

result = classifier(debate_text, candidate_topics)

print(result['labels'])
print(result['scores'])

Device set to use cpu


['education', 'healthcare', 'environment', 'Economy']
[0.2601376175880432, 0.2567405700683594, 0.24479518830776215, 0.2383265495300293]


In [5]:
import pandas as pd

pd.DataFrame([result['scores']], columns=result['labels'])

,education,healthcare,environment,Economy
0,0.260138,0.256741,0.244795,0.238327


In [9]:
import gradio as gr

iface = gr.Interface(fn=summarize, inputs="text", outputs="text", title="Text Summarization with T5", description="Enter text to get a summarized version using the T5 model.")
iface.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
